In [1]:
import sys, os
import pandas as pd
import numpy as np

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils

In [3]:
 

df=pd.read_csv('fer2013.csv')

In [4]:

X_train,train_y,X_test,test_y=[],[],[],[]

In [5]:
for index, row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
            X_train.append(np.array(val,'float32'))
            train_y.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
            X_test.append(np.array(val,'float32'))
            test_y.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")

In [6]:
num_features = 64
num_labels = 7
batch_size = 64
epochs = 30
width, height = 48, 48

In [7]:
X_train = np.array(X_train,'float32')
train_y = np.array(train_y,'float32')
X_test = np.array(X_test,'float32')
test_y = np.array(test_y,'float32')

In [8]:
train_y=np_utils.to_categorical(train_y, num_classes=num_labels)
test_y=np_utils.to_categorical(test_y, num_classes=num_labels)

In [9]:

X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)

X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

In [10]:

model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))

model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

In [11]:

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))

model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

In [12]:

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))

model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

In [13]:
model.add(Flatten())

In [14]:

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

In [15]:
model.add(Dense(num_labels, activation='softmax'))

In [16]:

model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

In [17]:

model.fit(X_train, train_y,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, test_y),
          shuffle=True)

Epoch 1/30
449/449 [==============================] - 344s 767ms/step - loss: 1.7267 - accuracy: 0.2882 - val_loss: 1.5544 - val_accuracy: 0.3828
Epoch 2/30
449/449 [==============================] - 493s 1s/step - loss: 1.5187 - accuracy: 0.4022 - val_loss: 1.3962 - val_accuracy: 0.4614
Epoch 3/30
449/449 [==============================] - 499s 1s/step - loss: 1.4110 - accuracy: 0.4549 - val_loss: 1.3422 - val_accuracy: 0.4745
Epoch 4/30
449/449 [==============================] - 519s 1s/step - loss: 1.3466 - accuracy: 0.4820 - val_loss: 1.2991 - val_accuracy: 0.4946
Epoch 5/30
449/449 [==============================] - 525s 1s/step - loss: 1.3020 - accuracy: 0.5011 - val_loss: 1.2814 - val_accuracy: 0.5065
Epoch 6/30
449/449 [==============================] - 505s 1s/step - loss: 1.2689 - accuracy: 0.5104 - val_loss: 1.2321 - val_accuracy: 0.5222
Epoch 7/30
449/449 [==============================] - 521s 1s/step - loss: 1.2323 - accuracy: 0.5290 - val_loss: 1.2249 - val_accuracy: 0.5

In [18]:

fer_json = model.to_json()
with open("fer.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer.h5")

In [19]:
import os
import cv2
import numpy as np
from keras.models import model_from_json
from keras.preprocessing import image

In [20]:

model = model_from_json(open("fer.json", "r").read())

model.load_weights('fer.h5')

In [21]:
face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [22]:
cap=cv2.VideoCapture(0)

In [ ]:
while True:
    ret,test_img=cap.read()
    if not ret:
        continue
    gray_img= cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)

    faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)


    for (x,y,w,h) in faces_detected:
        cv2.rectangle(test_img,(x,y),(x+w,y+h),(255,0,0),thickness=7)
        roi_gray=gray_img[y:y+w,x:x+h]
        roi_gray=cv2.resize(roi_gray,(48,48))
        img_pixels = image.img_to_array(roi_gray)
        img_pixels = np.expand_dims(img_pixels, axis = 0)
        img_pixels /= 255

        predictions = model.predict(img_pixels)

        
        max_index = np.argmax(predictions[0])

        emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
        predicted_emotion = emotions[max_index]

        cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

    resized_img = cv2.resize(test_img, (1000, 700))
    cv2.imshow('Facial emotion analysis ',resized_img)



    if cv2.waitKey(10) == ord('q'):#wait until 'q' key is pressed
        break

cap.release()
cv2.destroyAllWindows